In [ ]:
!pip install darts
import pandas as pd
import seaborn as sns
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv("/kaggle/input/playground-series-s3e20/train.csv").set_index("ID_LAT_LON_YEAR_WEEK")
final_df = pd.read_csv("/kaggle/input/playground-series-s3e20/test.csv")
df

# Feature Engineering

In [ ]:
df["loc"] = df["latitude"].astype("string")+"-"+ df["longitude"].astype("string")
df=df.drop(columns=["latitude","longitude"])
df['month'] = pd.to_datetime(df['year'].astype("string")+"_"+df['week_no'].astype("string")+"_1", format="%Y_%W_%w").dt.month
locs=df["loc"].drop_duplicates().to_frame()["loc"]


In [ ]:
target = df['emission']
features = df.drop(columns="emission")

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
rb = RobustScaler()
si = SimpleImputer()

features.loc[:,"SulphurDioxide_SO2_column_number_density":"Cloud_solar_zenith_angle"]=si.fit_transform(features.loc[:,"SulphurDioxide_SO2_column_number_density":"Cloud_solar_zenith_angle"])


In [ ]:
new_ft = target.to_frame()
# new_ft["year"] = valid["year"]
new_ft["diff_NO2"]=features["NitrogenDioxide_tropospheric_NO2_column_number_density"]-features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
new_ft["ratio_NO2"]=features["NitrogenDioxide_tropospheric_NO2_column_number_density"]/features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
new_ft["sum_NO2"]=features["NitrogenDioxide_tropospheric_NO2_column_number_density"]+features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
new_ft["sub_NO2"]=new_ft["sum_NO2"]-features["NitrogenDioxide_NO2_column_number_density"]
# new_ft["15_NO2_diff"]=features["NitrogenDioxide_NO2_column_number_density"]-features["NitrogenDioxide_NO2_column_number_density_15km"]
# new_ft["15_NO2_ratio"]=features["NitrogenDioxide_NO2_column_number_density"]/features["NitrogenDioxide_NO2_column_number_density_15km"]
new_ft["p_diff_NO2"]=new_ft["diff_NO2"] / features["NitrogenDioxide_tropopause_pressure"]
new_ft["p_ratio_NO2"]=new_ft["ratio_NO2"] / features["NitrogenDioxide_tropopause_pressure"]

new_ft["total_NO2"]=features["NitrogenDioxide_sensor_altitude"] * features["NitrogenDioxide_NO2_column_number_density"]
new_ft["total_slan_NO2"]=features["NitrogenDioxide_sensor_altitude"]  * features["NitrogenDioxide_NO2_slant_column_number_density"]



# new_ft["diff_NO2"]=features["H"]-features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
# new_ft["ratio_NO2"]=features["NitrogenDioxide_tropospheric_NO2_column_number_density"]/features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
# new_ft["sum_NO2"]=features["NitrogenDioxide_tropospheric_NO2_column_number_density"]+features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
# new_ft["sub_NO2"]=new_ft["sum_NO2"]-features["NitrogenDioxide_NO2_column_number_density"]
# new_ft["15_CO_diff"]=features["CarbonMonoxide_CO_column_number_density"]-features["C"]
# new_ft["15_CO_ratio"]=features["NitrogenDioxide_NO2_column_number_density"]/features["NitrogenDioxide_NO2_column_number_density_15km"]
# new_ft["p_diff_NO2"]=new_ft["diff_NO2"] / features["NitrogenDioxide_tropopause_pressure"]
# new_ft["p_ratio_NO2"]=new_ft["ratio_NO2"] / features["NitrogenDioxide_tropopause_pressure"]
new_ft["total_CO"]=features["CarbonMonoxide_sensor_altitude"] * features["CarbonMonoxide_CO_column_number_density"]
new_ft["total_slan_CO"]=features["CarbonMonoxide_sensor_altitude"]  * features["CarbonMonoxide_H2O_column_number_density"]





# new_ft["diff_SO2"]=features["S"]-features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
# new_ft["ratio_NO2"]=features["NitrogenDioxide_tropospheric_NO2_column_number_density"]/features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
# new_ft["sum_NO2"]=features["NitrogenDioxide_tropospheric_NO2_column_number_density"]+features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
# new_ft["sub_NO2"]=new_ft["sum_NO2"]-features["NitrogenDioxide_NO2_column_number_density"]
new_ft["15_SO2_diff"]=features["SulphurDioxide_SO2_column_number_density"]-features["SulphurDioxide_SO2_column_number_density_15km"]
new_ft["15_SO2_ratio"]=features["SulphurDioxide_SO2_column_number_density"]/features["SulphurDioxide_SO2_column_number_density_15km"]
# new_ft["p_diff_NO2"]=new_ft["diff_NO2"] / features["NitrogenDioxide_tropopause_pressure"]
# new_ft["p_ratio_NO2"]=new_ft["ratio_NO2"] / features["NitrogenDioxide_tropopause_pressure"]

# new_ft["total_NO2"]=features["NitrogenDioxide_sensor_altitude"] * features["NitrogenDioxide_NO2_column_number_density"]
# new_ft["total_slan_NO2"]=features["NitrogenDioxide_sensor_altitude"]  * features["NitrogenDioxide_NO2_slant_column_number_density"]









new_ft["cc_height_diff"] = features["Cloud_cloud_top_height"] -  features["Cloud_cloud_base_height"]  
new_ft["cc_p_ratio"] = features["Cloud_cloud_top_pressure"]/  features["Cloud_cloud_base_pressure"]  

# new_ft["derive_cloud_base_poressure"] = derive["shifted_Cloud_cloud_base_pressure"]
new_ft=new_ft.drop(columns=["emission"])
ssi = SimpleImputer(missing_values=np.nan)

from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(2)
poly=pd.DataFrame(pf.fit_transform(pd.DataFrame(ssi.fit_transform(new_ft.replace(-np.inf,np.nan)),columns=new_ft.columns)),columns=pf.get_feature_names_out(),index=new_ft.index).join(target)
# for i,col in enumerate(poly):
#     plt.subplot()
#     sns.scatterplot(data=poly,x=col,y="emission")
#     plt.show()
nc = ["cc_p_ratio","total_slan_NO2 total_slan_CO","sub_NO2 total_slan_CO","total_CO cc_height_diff","cc_height_diff cc_p_ratio","total_CO total_slan_CO"]
features =poly[nc].join(features[["loc","year","week_no","month"]])

In [ ]:
features.loc[:,"cc_p_ratio":"total_CO total_slan_CO"]=rb.fit_transform(features.loc[:,"cc_p_ratio":"total_CO total_slan_CO"])

In [ ]:
joined = features.join(target)
quantiles = joined.groupby("loc")["emission"].quantile(0.55).to_frame().rename(columns={"emission":"quantiles"})
updated_em = joined.join(quantiles,on="loc").query("((year == 2019 and week_no > 40) or (year<= 2021 and week_no < 50 ))").assign(new_emission=lambda d: d["emission"] *(d["emission"] >= d["quantiles"]) +  d["quantiles"]*(d["emission"] < d["quantiles"])  )

In [ ]:
target.loc[updated_em.index] = updated_em["new_emission"]

# Model Training

In [ ]:
from darts import TimeSeries
from sklearn.svm import SVR,NuSVR
from darts.models import XGBModel,CatBoostModel,RegressionModel,RandomForest,XGBModel,ExponentialSmoothing
from sklearn.linear_model import ElasticNet,LinearRegression,Lasso
from darts.utils.utils import ModelMode,SeasonalityMode
from darts.metrics import rmse,mape
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
ttmp = 0
ttres = 0
n = 497
targetd = None
ftd = None
# def gen_features():
#     jj = joined[joined["loc"] == loc]
#     dc = {}
#     dc["ind"]=pd.RangeIndex(stop=159)
#     # dc['r3'] = jj['CarbonMonoxide_CO_column_number_density'] * jj['NitrogenDioxide_NO2_slant_column_number_density']
#     # dc['r1']= dc['r3'] - (jj['Cloud_cloud_optical_depth'] * jj["Cloud_surface_albedo"])
#     dc["d"]= (jj['CarbonMonoxide_CO_column_number_density']-jj['CarbonMonoxide_CO_column_number_density'].shift(-1)).backfill()
#     # r2=TimeSeries.from_series(jj["SulphurDioxide_SO2_slant_column_number_density"] / jj["NitrogenDioxide_NO2_slant_column_number_density"])
#     return pd.DataFrame(dc)

for i,loc in enumerate(locs.iloc[0:n]):
    selected_ft = features.query(f"loc == '{loc}'")
    selected_ft.loc[:,"ind"]=pd.RangeIndex(stop=159)
    # ftd_new = TimeSeries.from_dataframe(selected_ft.drop(columns="loc"),time_col ="ind")
    ftd_new = selected_ft.drop(columns=["loc","ind"]).set_index(["year","week_no","month"])
    ftd_new.columns+="_"+str(i)
    if ftd == None:
        ftd = [ftd_new]
    else: 
        ftd.append(ftd_new)

ftd=pd.concat(ftd,axis=1).astype("float")
ftd=ftd.reset_index()
print((ftd.columns,len(ftd.columns)))
pca_rb = RobustScaler()
pca = PCA()
ftd = pd.DataFrame(pca.fit_transform(pca_rb.fit_transform(ftd))[:,:30],columns=pca.get_feature_names_out()[:30])
ftd.loc[:,"ind"]=pd.RangeIndex(stop=159)
ftd_test = ftd

ftd = TimeSeries.from_dataframe(ftd,time_col ="ind")
    

for loc in locs.iloc[0:n]:
    selected_ft = features.query(f"loc == '{loc}'")
    selected_ft.loc[:,"ind"]=pd.RangeIndex(stop=159)
    targetd_new = TimeSeries.from_dataframe(selected_ft.join(target)[["emission","ind"]],time_col="ind")
    targetd = targetd_new if targetd == None else targetd.stack(targetd_new)




In [ ]:
md_1 = RandomForest(lags=1,output_chunk_length=49,lags_past_covariates=2,lags_future_covariates=[2]*len(ftd.columns)) # Gut
md_1.fit(targetd,past_covariates=targetd,future_covariates=ftd)
# md_1=pd.read_pickle("/kaggle/working/md_1.model")

In [ ]:
# md_2 = RegressionModel(lags=3,output_chunk_length=49,model=NuSVR(),lags_past_covariates=5,lags_future_covariates=[2]*len(ftd.columns))
# md_2.fit(targetd,past_covariates=targetd,future_covariates=ftd)



In [ ]:
# import pickle
# pickle.dump(md_2, open("/kaggle/working/md_2.model", 'wb'))

In [ ]:
md_3 = RegressionModel(lags=3,output_chunk_length=49,model=LinearRegression(),lags_past_covariates=5,lags_future_covariates=[5]*len(ftd.columns))
md_3.fit(targetd,past_covariates=targetd,future_covariates=ftd)

In [ ]:
valid=final_df.set_index("ID_LAT_LON_YEAR_WEEK")
valid["loc"] = valid["latitude"].astype("string")+"-"+ valid["longitude"].astype("string")
valid['month'] = pd.to_datetime(valid['year'].astype("string")+"_"+valid['week_no'].astype("string")+"_1", format="%Y_%W_%w").dt.month
valid=valid.drop(columns=["latitude","longitude"])
valid.loc[:,"SulphurDioxide_SO2_column_number_density":"Cloud_solar_zenith_angle"]=si.transform(valid.loc[:,"SulphurDioxide_SO2_column_number_density":"Cloud_solar_zenith_angle"])


In [ ]:
# new_ft["year"] = features["year"]
new_ft = pd.DataFrame()
new_ft["diff_NO2"]=valid["NitrogenDioxide_tropospheric_NO2_column_number_density"]-valid["NitrogenDioxide_stratospheric_NO2_column_number_density"]
new_ft.index = valid.index
new_ft["ratio_NO2"]=valid["NitrogenDioxide_tropospheric_NO2_column_number_density"]/valid["NitrogenDioxide_stratospheric_NO2_column_number_density"]
new_ft["sum_NO2"]=valid["NitrogenDioxide_tropospheric_NO2_column_number_density"]+valid["NitrogenDioxide_stratospheric_NO2_column_number_density"]
new_ft["sub_NO2"]=new_ft["sum_NO2"]-valid["NitrogenDioxide_NO2_column_number_density"]
# new_ft["15_NO2_diff"]=features["NitrogenDioxide_NO2_column_number_density"]-features["NitrogenDioxide_NO2_column_number_density_15km"]
# new_ft["15_NO2_ratio"]=features["NitrogenDioxide_NO2_column_number_density"]/features["NitrogenDioxide_NO2_column_number_density_15km"]
new_ft["p_diff_NO2"]=new_ft["diff_NO2"] / valid["NitrogenDioxide_tropopause_pressure"]
new_ft["p_ratio_NO2"]=new_ft["ratio_NO2"] / valid["NitrogenDioxide_tropopause_pressure"]

new_ft["total_NO2"]=valid["NitrogenDioxide_sensor_altitude"] * valid["NitrogenDioxide_NO2_column_number_density"]
new_ft["total_slan_NO2"]=valid["NitrogenDioxide_sensor_altitude"]  * valid["NitrogenDioxide_NO2_slant_column_number_density"]



# new_ft["diff_NO2"]=features["H"]-features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
# new_ft["ratio_NO2"]=features["NitrogenDioxide_tropospheric_NO2_column_number_density"]/features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
# new_ft["sum_NO2"]=features["NitrogenDioxide_tropospheric_NO2_column_number_density"]+features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
# new_ft["sub_NO2"]=new_ft["sum_NO2"]-features["NitrogenDioxide_NO2_column_number_density"]
# new_ft["15_CO_diff"]=features["CarbonMonoxide_CO_column_number_density"]-features["C"]
# new_ft["15_CO_ratio"]=features["NitrogenDioxide_NO2_column_number_density"]/features["NitrogenDioxide_NO2_column_number_density_15km"]
# new_ft["p_diff_NO2"]=new_ft["diff_NO2"] / features["NitrogenDioxide_tropopause_pressure"]
# new_ft["p_ratio_NO2"]=new_ft["ratio_NO2"] / features["NitrogenDioxide_tropopause_pressure"]
new_ft["total_CO"]=valid["CarbonMonoxide_sensor_altitude"] * valid["CarbonMonoxide_CO_column_number_density"]
new_ft["total_slan_CO"]=valid["CarbonMonoxide_sensor_altitude"]  * valid["CarbonMonoxide_H2O_column_number_density"]





# new_ft["diff_SO2"]=features["S"]-features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
# new_ft["ratio_NO2"]=features["NitrogenDioxide_tropospheric_NO2_column_number_density"]/features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
# new_ft["sum_NO2"]=features["NitrogenDioxide_tropospheric_NO2_column_number_density"]+features["NitrogenDioxide_stratospheric_NO2_column_number_density"]
# new_ft["sub_NO2"]=new_ft["sum_NO2"]-features["NitrogenDioxide_NO2_column_number_density"]
new_ft["15_SO2_diff"]=valid["SulphurDioxide_SO2_column_number_density"]-valid["SulphurDioxide_SO2_column_number_density_15km"]
new_ft["15_SO2_ratio"]=valid["SulphurDioxide_SO2_column_number_density"]/valid["SulphurDioxide_SO2_column_number_density_15km"]
# new_ft["p_diff_NO2"]=new_ft["diff_NO2"] / features["NitrogenDioxide_tropopause_pressure"]
# new_ft["p_ratio_NO2"]=new_ft["ratio_NO2"] / features["NitrogenDioxide_tropopause_pressure"]

# new_ft["total_NO2"]=features["NitrogenDioxide_sensor_altitude"] * features["NitrogenDioxide_NO2_column_number_density"]
# new_ft["total_slan_NO2"]=features["NitrogenDioxide_sensor_altitude"]  * features["NitrogenDioxide_NO2_slant_column_number_density"]









new_ft["cc_height_diff"] = valid["Cloud_cloud_top_height"] -  valid["Cloud_cloud_base_height"]  
new_ft["cc_p_ratio"] = valid["Cloud_cloud_top_pressure"]/  valid["Cloud_cloud_base_pressure"]  

# new_ft["derive_cloud_base_poressure"] = derive["shifted_Cloud_cloud_base_pressure"]
poly=pd.DataFrame(pf.transform(pd.DataFrame(ssi.transform(new_ft.replace(-np.inf,np.nan)),columns=new_ft.columns)),columns=pf.get_feature_names_out(),index=new_ft.index)
# for i,col in enumerate(poly):
#     plt.subplot()
#     sns.scatterplot(data=poly,x=col,y="emission")
#     plt.show()
nc = ["cc_p_ratio","total_slan_NO2 total_slan_CO","sub_NO2 total_slan_CO","total_CO cc_height_diff","cc_height_diff cc_p_ratio","total_CO total_slan_CO"]
valid =poly[nc].join(valid[["loc","year","week_no","month"]])
valid.loc[:,"cc_p_ratio":"total_CO total_slan_CO"]=rb.transform(valid.loc[:,"cc_p_ratio":"total_CO total_slan_CO"])

In [ ]:
ftd_valid = None
for i,loc in enumerate(locs.iloc[0:n]):
    selected_ft = valid.query(f"loc == '{loc}'")
    selected_ft.loc[:,"ind"]=pd.RangeIndex(stop=49)
    # ftd_new = TimeSeries.from_dataframe(selected_ft.drop(columns="loc"),time_col ="ind")
    ftd_new = selected_ft.drop(columns=["loc","ind"]).set_index(["year","week_no","month"])
    ftd_new.columns+="_"+str(i)
    if ftd_valid == None:
        ftd_valid = [ftd_new]
    else: 
        ftd_valid.append(ftd_new)
        
ftd_valid=pd.concat(ftd_valid,axis=1).astype("float")
ftd_valid=ftd_valid.reset_index()


ftd_valid = pd.DataFrame(pca.transform(pca_rb.transform(ftd_valid))[:,:30],columns=pca.get_feature_names_out()[:30])
# ftd_valid = TimeSeries.from_dataframe(selected_ft.drop(columns="loc"),time_col ="ind")

    

In [ ]:
ftd_valid

In [ ]:
f_cov=pd.concat([ftd_test,ftd_valid])
f_cov.loc[:,"ind"]=pd.RangeIndex(stop=208)
f_cov = TimeSeries.from_dataframe(f_cov,time_col ="ind")


In [ ]:
f_cov

In [ ]:
res_1=md_1.predict(49,past_covariates=targetd,future_covariates=f_cov)
# res_2=md_2.predict(49,past_covariates=targetd,future_covariates=ftd_valid)
res_2=0
res_3=md_3.predict(49,past_covariates=targetd,future_covariates=f_cov)

In [ ]:
res_3

In [ ]:
x = [ 1.19946351,0, -0.17475927]
pred = float(x[0])*res_1 + float(x[1])*res_2 +float(x[2])*res_3

In [ ]:
pred = res_1

# Integrate Solution from discussion

In [ ]:
import pandas as pd
import numpy as np

# Read data
PATH = "/kaggle/input/playground-series-s3e20/"
train = pd.read_csv(PATH + "train.csv",index_col="ID_LAT_LON_YEAR_WEEK")
test = pd.read_csv(PATH  + "test.csv",index_col="ID_LAT_LON_YEAR_WEEK")
sample = pd.read_csv(PATH + "sample_submission.csv")

# Create new column with combined lat/lon
train.insert(2,"lat_lon", list(zip(train["latitude"],train["longitude"])))

# Location of each station
locations = train["lat_lon"].unique()

# Function to get emissions for specified year and location. Only use first 49 weeks.
def get_emissions_loc_year( loc, year ):
    df = train[(train["lat_lon"]==loc) & (train["year"]==year) & (train["week_no"]<49)].copy()
    return df["emission"].values

# Function to get the max emission (over 3 years) at location.
def get_emissions_max( loc ):
    emiss2019 = get_emissions_loc_year(loc,2019)
    emiss2020 = get_emissions_loc_year(loc,2020)
    emiss2021 = get_emissions_loc_year(loc,2021)
    return np.max([emiss2019,emiss2020,emiss2021],axis=0)
         
# Create 2022 predictions
predictions_acc = []
for loc in locations:
    emission = get_emissions_max( loc )
    predictions_acc.append( emission )
  
predictions_2022 = np.hstack(predictions_acc)
    
# Create submission
submission = sample.copy()
submission["emission"] = predictions_2022
submission.to_csv("submission_23.csv",index=False)


In [ ]:
export = pd.DataFrame()
final_output = pd.DataFrame({"ID_LAT_LON_YEAR_WEEK":[],"Emission": []})
for i in range(497):
    loc = locs[i]
    ind=valid[valid["loc"] ==  locs[i]].index
    pr=pred[pred.components[i]].pd_dataframe().iloc[:,0].values
    final_output = pd.concat([final_output,pd.DataFrame({"ID_LAT_LON_YEAR_WEEK":ind,"Emission": pr})])

In [ ]:
final_output=final_output.set_index("ID_LAT_LON_YEAR_WEEK")

In [ ]:
final_mixed = submission.set_index("ID_LAT_LON_YEAR_WEEK").join(final_output)

In [ ]:
final_mixed.describe()

## Merging Solutions

In [ ]:
fiiinal=(2*final_mixed["emission"] + final_mixed["Emission"])/3
fiiinal.to_frame().rename(columns={0:"Emission"}).to_csv("mix_23_rnd.csv")
fiiinal.describe()